# Rice Leaf Disease Detection - Model Evaluation
This notebook evaluates multiple transformer-based models for rice leaf disease classification.

In [ ]:

import os
import json
import torch
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoModelForImageClassification, AutoProcessor
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive


## Function: Model Evaluation
This function loads a pre-trained model and evaluates it on the test dataset.

In [ ]:

def evaluate_model(model_name, dataset, labels, batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu"):
    """Loads a model and evaluates it on the dataset in batches."""
    print(f"Evaluating {model_name}...")

    model = AutoModelForImageClassification.from_pretrained(model_name).to(device)
    processor = AutoProcessor.from_pretrained(model_name)

    y_true, y_pred = [], []
    all_images, all_labels = [], []

    start_time = time.time()

    # Load all images and labels first
    for example in tqdm(dataset, desc=f"Loading {model_name}"):
        all_images.append(example["image"])
        all_labels.append(example["label"])

    # Process images in batches
    for i in tqdm(range(0, len(all_images), batch_size), desc=f"Testing {model_name}"):
        batch_images = all_images[i:i+batch_size]
        batch_labels = all_labels[i:i+batch_size]

        inputs = processor(images=batch_images, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            batch_preds = torch.argmax(outputs.logits, dim=-1).cpu().tolist()

        y_true.extend(batch_labels)
        y_pred.extend(batch_preds)

    elapsed_time = time.time() - start_time
    print(f"Model {model_name} evaluation completed in {elapsed_time:.2f} seconds.")
    return y_true, y_pred, elapsed_time


## Function: Generate Report
This function generates and saves a classification report and confusion matrix.

In [ ]:

def generate_report(y_true, y_pred, labels, model_name, output_dir, elapsed_time):
    """Generates and saves classification report and confusion matrix."""
    model_safe_name = model_name.split("/")[-1] + "-tl"

    report = classification_report(y_true, y_pred, target_names=labels, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype("float") / cm.sum(axis=1, keepdims=True)

    report["evaluation_time_sec"] = elapsed_time

    # Save JSON report
    report_path = os.path.join(output_dir, f"{model_safe_name}_report.json")
    with open(report_path, "w") as f:
        json.dump(report, f, indent=4)

    # Save Excel report
    report_df = pd.DataFrame(report).transpose()
    excel_path = os.path.join(output_dir, f"{model_safe_name}_report.xlsx")

    with pd.ExcelWriter(excel_path) as writer:
        report_df.to_excel(writer, sheet_name="Classification Report")
        pd.DataFrame(cm, index=labels, columns=labels).to_excel(writer, sheet_name="Confusion Matrix")
        pd.DataFrame(cm_normalized, index=labels, columns=labels).to_excel(writer, sheet_name="Normalized Confusion Matrix")

    # Save confusion matrix plot
    def save_cm_plot(matrix, title, filename, fmt="d"):
        plt.figure(figsize=(10, 8))
        sns.heatmap(matrix, annot=True, fmt=fmt, cmap="Blues", xticklabels=labels, yticklabels=labels)
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.title(title, pad=20)
        plt.xticks(rotation=30)
        plt.yticks(rotation=30)
        plt.savefig(os.path.join(output_dir, filename), bbox_inches="tight", pad_inches=0.5)
        plt.close()

    save_cm_plot(cm, f"{model_safe_name} Confusion Matrix", f"{model_safe_name}_confusion_matrix.png")
    save_cm_plot(cm_normalized, "Normalized Confusion Matrix", f"{model_safe_name}_normalized_confusion_matrix.png", fmt=".2f")


## Main Function
This function loads the dataset, evaluates models, and saves reports to Google Drive.

In [ ]:

def main():
    """Mounts Google Drive, loads dataset, evaluates models, and saves reports."""
    drive.mount("/content/drive")

    models_path = [
        "cvmil/resnet-50_rice-leaf-disease-augmented_tl",
        "cvmil/vit-base-patch16-224_rice-leaf-disease-augmented_tl",
        "cvmil/swin-base-patch4-window7-224_rice-leaf-disease-augmented_tl",
        "cvmil/deit-base-patch16-224_rice-leaf-disease-augmented_tl",
        "cvmil/beit-base-patch16-224_rice-leaf-disease-augmented_tl",
        "cvmil/dinov2-base_rice-leaf-disease-augmented_tl",
    ]

    dataset = load_dataset("cvmil/rice-leaf-disease-augmented", split="test")
    labels = dataset.features["label"].names

    output_dir = "/content/drive/Shareddrives/CS198-Drones/test_tl_output/"
    os.makedirs(output_dir, exist_ok=True)

    for model_name in models_path:
        try:
            y_true, y_pred, elapsed_time = evaluate_model(model_name, dataset, labels)
            generate_report(y_true, y_pred, labels, model_name, output_dir, elapsed_time)
        except Exception as e:
            print(f"⚠️ Error processing {model_name}: {e}")

    print("✅ Evaluation completed. Reports saved to Google Drive.")

if __name__ == "__main__":
    main()
